In [1]:
import numpy as np
from scipy.io import arff
import pandas as pd
import time

In [2]:
#Load dataset

Natt = 14 # Numero de atributos
data = arff.loadarff('yeast/yeast-train.arff')
df = pd.DataFrame(data[0])
data_train = df.to_numpy()
X_train = data_train[:,:-Natt]
Y_train = data_train[:,-Natt:].astype(np.int)

data = arff.loadarff('yeast/yeast-test.arff')
df = pd.DataFrame(data[0])
data_test = df.to_numpy()
X_test = data_test[:,:-Natt]
Y_test = data_test[:,-Natt:].astype(np.int)

In [8]:
class kNN:

    def __init__(self,k,X_train,X_test):
        self.k = k
        self.X_train = X_train
        self.X_test = X_test
        self.N_train = len(X_train)
        self.N_test = len(X_test)
    
    def train(self):
    
        # Calculo das distancias
        # dist(i,j) = distancia entre a i-esima amostra de teste e a j-esima amostra de treinamento
        dist = np.zeros((self.N_test,self.N_train)) 

        for i in np.arange(self.N_test):
            for j in np.arange(self.N_train):
                # Distancia euclidiana
                dist[i,j] = np.sqrt(np.sum((self.X_test[i] - self.X_train[j])**2))

        self.ind_neigh = np.argsort(dist,axis=1) # indices dos vizinhos em ordem crescente de distancia
        dist_neigh = np.array([dist[i,self.ind_neigh[i]] for i in range(self.N_test)]) # distancias ordenadas
        self.k_dist = dist_neigh[:,:k] # distancias dos k-vizinhos mais proximos a cada amostra

    def predict(self,Y_train):
        
        Nclasses = int(Y_train.max()) + 1
        classes_neigh = Y_train[self.ind_neigh] # classes dos vizinhos mais proximos
        k_classes = classes_neigh[:,:k] # classes dos k-vizinhos mais proximos a cada amostra

        votes = np.zeros((self.N_test,Nclasses)) # votos das classes para cada amostra

        # soma de votos para cada classe
        for c in np.arange(Nclasses):
            votes[:,c] = votes[:,c] + ((k_classes == c)/(self.k_dist+1)).sum(axis=1) # votos ponderados pelo inverso da distancia+1

        # retorna classe com maior voto para cada amostra
        return votes.argmax(axis=1)

In [4]:
def eval_MLL(pred,Y_test):
    
    Nsamples,Nlabels = Y_test.shape
    hl = np.sum(abs(Y_test - pred))/(Nsamples*Nlabels)
    acc = (Y_test*pred).sum()/((np.logical_or(Y_test,pred)).sum())
    prec = (Y_test*pred).sum()/(pred.sum())
    recall = (Y_test*pred).sum()/(Y_test.sum())
    f1 = 2*prec*recall/(prec+recall)
    
    return hl,acc,prec,recall,f1

In [9]:
k = 12
Nclasses = 2 #Numero de classes

j = 0
Nsamples,Nlabels = Y_test.shape
pred = np.zeros((Nsamples,Nlabels))

# Treinamento

print('Iniciando treinamento do classificador...')
start = time.time()
clf = kNN(k,X_train,X_test)
clf.train()
end = time.time()
print('Treinamento do classificador finalizado em %f segundos' %((end-start)))

Iniciando treinamento do classificador...
Treinamento do classificador finalizado em 29.495472 segundos


In [10]:
# Predicoes

for j in np.arange(Nlabels):
    
    pred[:,j] = clf.predict(Y_train[:,j])
    
hl,acc,prec,recall,f1 = eval_MLL(pred,Y_test)

print ('Hamming Loss = %f, F1 = %f' %(hl,f1))

Hamming Loss = 0.202368, F1 = 0.635930
